We will create a deep neural network with Keras.

In [1]:
# import some modules we may need
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# try out PCA as well as some feature selection. or try both!
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Activation

/Users/rajsingh/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# read in the csv file and put it in a DataFrame.
df_train = pd.read_csv('audio_training_data_cleaned.csv')

In [3]:
# drop any null values we may have forgotten
df_train = df_train.dropna(how='any',axis=0)

In [4]:
# split into X_train and y_train
X_train = df_train.drop(columns=['filename','age','Unnamed: 0']).values
y_train = df_train['age']

# to do feature selection, must convert categorical values to
# numerical values
replaced = {'teens':1,'twenties':2,'thirties':3,'fourties':4,'fifties':5,'sixties':6,
            'seventies':7,'eighties':8}
y_train_encoded = y_train.replace(replaced)

In [5]:
%%time
# use LASSO technique to do some feature selection because there are 170 features
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel

lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(X_train, y_train_encoded)
model = SelectFromModel(lsvc, prefit=True)
X_train = model.transform(X_train)

CPU times: user 1min 51s, sys: 1.62 s, total: 1min 53s
Wall time: 2min


In [6]:
%%time
# try normal 3-layer perceptron
from keras.utils import to_categorical
from keras import regularizers
from keras.layers import Dropout
y_train_keras_encoded = to_categorical(y_train_encoded)[:,1:] # adds unneeded extra column

DL_model = Sequential()
DL_model.add(Dropout(0.15))
DL_model.add(Dense(256, activation='relu', input_shape = (X_train.shape[1],)))

DL_model.add(Dense(1024, activation='relu'))

DL_model.add(Dense(1024, activation='relu'))

DL_model.add(Dense(256, activation='relu'))

DL_model.add(Dense(8, activation='softmax'))

DL_model.compile(optimizer='adam',loss='categorical_crossentropy',
             metrics=['accuracy'])
DL_model.fit(X_train,y_train_keras_encoded,epochs=100,batch_size=100)

Epoch 1/100
73765/73765 [==============================] - 33s 447us/step - loss: 1.7663 - acc: 0.3073
Epoch 2/100
73765/73765 [==============================] - 28s 373us/step - loss: 1.6860 - acc: 0.3245
Epoch 3/100
73765/73765 [==============================] - 31s 415us/step - loss: 1.6555 - acc: 0.3392
Epoch 4/100
73765/73765 [==============================] - 31s 420us/step - loss: 1.6202 - acc: 0.3528
Epoch 5/100
73765/73765 [==============================] - 32s 430us/step - loss: 1.5880 - acc: 0.3681
Epoch 6/100
73765/73765 [==============================] - 29s 398us/step - loss: 1.5493 - acc: 0.3871
Epoch 7/100
73765/73765 [==============================] - 29s 398us/step - loss: 1.5100 - acc: 0.4033
Epoch 8/100
73765/73765 [==============================] - 29s 398us/step - loss: 1.4640 - acc: 0.4223
Epoch 9/100
73765/73765 [==============================] - 30s 402us/step - loss: 1.4176 - acc: 0.4464
Epoch 10/100
73765/73765 [==============================] - 30s 402us/ste

73765/73765 [==============================] - 38s 511us/step - loss: 0.5894 - acc: 0.7868
Epoch 80/100
73765/73765 [==============================] - 33s 451us/step - loss: 0.5847 - acc: 0.7897
Epoch 81/100
73765/73765 [==============================] - 31s 414us/step - loss: 0.5832 - acc: 0.7895
Epoch 82/100
73765/73765 [==============================] - 32s 429us/step - loss: 0.5797 - acc: 0.7913
Epoch 83/100
73765/73765 [==============================] - 27s 370us/step - loss: 0.5766 - acc: 0.7917
Epoch 84/100
73765/73765 [==============================] - 30s 400us/step - loss: 0.5673 - acc: 0.7951
Epoch 85/100
73765/73765 [==============================] - 31s 425us/step - loss: 0.5709 - acc: 0.7960
Epoch 86/100
73765/73765 [==============================] - 25s 339us/step - loss: 0.5638 - acc: 0.7979
Epoch 87/100
73765/73765 [==============================] - 25s 336us/step - loss: 0.5686 - acc: 0.7962
Epoch 88/100
73765/73765 [==============================] - 26s 351us/step - 

In [7]:
# now we need to test data against the testing set
# import testing set
testing_df = pd.read_csv('audio_testing_data_cleaned.csv')
# drop any null values we may have forgotten
testing_df = testing_df.dropna(how='any',axis=0)

# import another testing set
other_testing_df = pd.read_csv('audio_other_testing_data_cleaned.csv')
# drop any null values we may have forgotten
other_testing_df = other_testing_df.dropna(how='any',axis=0)

In [8]:
X_test = testing_df.drop(columns=['filename','age','Unnamed: 0']).values
X_test = model.transform(X_test)
y_test = testing_df['age']
y_test_encoded = y_test.replace(replaced)
y_test_keras_encoded = to_categorical(y_test_encoded)[:,1:] # adds unneeded extra column

X_other_test = other_testing_df.drop(columns=['filename','age','Unnamed: 0']).values

X_other_test= model.transform(X_other_test)
y_other_test = other_testing_df['age']
y_other_test_encoded = y_other_test.replace(replaced)
y_other_test_keras_encoded = to_categorical(y_other_test_encoded)[:,1:]

In [9]:
%%time
score1 = DL_model.evaluate(X_test,y_test_keras_encoded,batch_size=32)
print("Results on primary testing set:")
print("Loss: {}".format(score1[0]))
print("Accuracy: {}".format(score1[1]))

1542/1542 [==============================] - 0s 201us/step
Results on primary testing set:
Loss: 0.755758075244
Accuracy: 0.730869001374
CPU times: user 558 ms, sys: 35 ms, total: 593 ms
Wall time: 320 ms
